# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai_client = OpenAI()
ollama_client = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
system_prompt = """You are a technical tutor and coding expert. When asked to explain code or technical concepts:
1. Break down the code step by step
2. Explain what each part does
3. Explain why someone would use this pattern
4. Provide context about when this would be useful
5. Keep explanations clear and educational
Respond in markdown format."""

In [ ]:
# here is the question; type over this to ask something new
question = """Please explain what this code does and why: yield from {book.get("author") for book in books if book.get("author")}"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming
def get_openai_response_streaming(question):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    
    print("GPT-4o-mini Response (Streaming):\n")
    
    display_handle = display(Markdown(""), display_id=True)
    response_text = ""
    
    # Get the display_id, handling the case where display_handle might be None
    display_id = getattr(display_handle, 'display_id', None) if display_handle else None
    
    stream = openai_client.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True
    )
    
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response_text += chunk.choices[0].delta.content
            if display_id:
                update_display(Markdown(response_text), display_id=display_id)
    
    return response_text

openai_response = get_openai_response_streaming(question)

In [ ]:
# Get Llama 3.2 to answer
def get_ollama_response(question):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    
    print("Llama 3.2 Response:\n")
    
    try:
        response = ollama_client.chat.completions.create(
            model=MODEL_LLAMA,
            messages=messages
        )
        
        response_content = response.choices[0].message.content
        display(Markdown(response_content))
        return response_content
        
    except Exception as e:
        error_msg = f"**Error connecting to Ollama:** {str(e)}\n\n"
        error_msg += "**Troubleshooting:**\n"
        error_msg += "1. Make sure Ollama is installed and running\n"
        error_msg += "2. Run `ollama serve` in a terminal\n"
        error_msg += "3. Run `ollama pull llama3.2` to download the model\n"
        error_msg += "4. Check that http://localhost:11434 is accessible"
        
        display(Markdown(error_msg))
        return None

ollama_response = get_ollama_response(question)